In [1]:
import pandas as pd
from google.cloud import storage
from google.cloud.storage.blob import Blob
import json
import pdb
from typing import List

# PROJECT_PREFIX = 'fish-species-classification'
BUCKET = 'data-and-predictions-usc1'
PREDICTIONS_PREFIX = 'fish-species-classification/test/prediction-fish-species-classification-2022-10-03T10:02:51.293695Z'

storage_client = storage.Client()

In [3]:
def download_all_files(bucket, prefix, storage_client) -> List[str]:
    blobs = storage_client.list_blobs(
        BUCKET,
        prefix=PREDICTIONS_PREFIX,
    )

    all_objects = []
    for blob in blobs:
        jsonl_predictions = blob.download_as_text()
        all_objects.append(jsonl_predictions)

    return all_objects


predictions_jsonl_all = download_all_files(bucket=BUCKET, prefix=PREDICTIONS_PREFIX, storage_client=storage_client)

In [9]:
def extract_predictions_from_jsonl(jsonl_str: str, dlq: List[str]) -> pd.DataFrame:
    pd_predictions = []
    for line in jsonl_str.split('\n'):
        try:
            record = json.loads(line)
            labels = record['prediction']['displayNames']
            confidences = record['prediction']['confidences']
            pd_predictions.append(
                pd.Series(confidences, index=labels)
            )
            # Add identifier
            image_path = record['instance']['jsonl_str']
            img_name = image_path.split('/').get(-1)
            labels.append('image')
            confidences.append(img_name)
        except:
            dlq.append(line)

    # df = pd.concat(predictions, axis='columns').T
    return pd_predictions, dlq
    
dlq = []
# p, dlq = extract_predictions_from_jsonl(jsonl_str=jsonl_str, dlq=dlq)
# len(p)

In [16]:
def extract_all_predictions(jsonl_predictions: str) -> pd.DataFrame:
    predictions_extracted_all = []
    dlq = []
    for jsonl_prediction in jsonl_predictions:
        predictions_extracted, dlq = extract_predictions_from_jsonl(jsonl_str=jsonl_prediction, dlq=dlq)
        predictions_extracted_all.extend(predictions_extracted)
    
    df = pd.concat(
        predictions_extracted_all, 
        axis='columns'
    ) \
    .T
    
    return df

df = extract_all_predictions(predictions_jsonl_all)
df

,NoF,BET,OTHER,LAG,ALB,SHARK,YFT,DOL
0,0.981931,0.007087,0.003372,0.003088,0.001757,0.000985,0.000960,0.000820
1,0.003020,0.002345,0.004051,0.003151,0.980238,0.001425,0.003926,0.001845
2,0.005592,0.010044,0.009778,0.004985,0.016388,0.009977,0.939918,0.003318
3,0.002172,0.002724,0.002211,0.001730,0.985127,0.001100,0.003779,0.001156
4,0.006399,0.013965,0.006030,0.002840,0.955483,0.003907,0.009002,0.002374
...,...,...,...,...,...,...,...,...
995,0.024960,0.013157,0.017409,0.007206,0.087423,0.034916,0.810338,0.004592
996,0.000476,0.015375,0.000044,0.728741,0.152693,0.000727,0.101912,0.000033
997,0.005226,0.003585,0.002989,0.001732,0.981581,0.001587,0.002169,0.001130
998,0.017083,0.027919,0.017161,0.006823,0.552932,0.006589,0.366660,0.004833


In [17]:
# sanity check
np.allclose(
    df.sum(axis='columns'),

    1

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
995    1.0
996    1.0
997    1.0
998    1.0
999    1.0
Length: 1000, dtype: float64